In [52]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torchvision import datasets, transforms
# import numpy as np
# Import main packages
import os
import copy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from typing import Optional, Callable
from sklearn.metrics import accuracy_score, f1_score
from sklearn.covariance import LedoitWolf
import json

import torchvision.transforms as transforms

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [80]:
class CoinDataset(Dataset):
    """DHMC dataset using 2 classes"""

    def __init__(self, features_path : str, label_path,train : bool = False) -> None:
        """
        Attributes:
            raw_data (list of dict): (M) List of M slides raw data as dictionaries. 
            train (bool): True if data are the training set. False otherwise
            
        Args:
            features_path (str): The path to the features file
            train (bool): Whether it is the training dataset or not
        """
        
        super().__init__()
        # Load raw data from path
        
        # self.raw_data = torch.load(features_path)
        # load labels from json file
        self.raw_data = []
        data_json = None
        with open(label_path, 'r') as f:
            data_json = json.load(f)
        for x in data_json:
            filename = x["filename"]
            img_path = f"{features_path}/{filename}"
            img = Image.open(img_path)

            mean=[0.485, 0.456, 0.406]
            std=[0.229, 0.224, 0.225]
            transform = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
            img = transform(img)
            # print(img_tr)
            # img = np.array(img_tr)
            # print(img)
            # img = img.flatten()
            self.raw_data.append({"image_features": img, "label": int(x["value"])})

            #open image
        # Set if training or not
        self.train = train

    def __len__(self) -> int:
        """Returns the length of the dataset

        Returns:
            int: The length M of the dataset
        """

        n_data = 0
        
        # ------------------
        # Your code here ... 
        # ------------------
        n_data = len(self.raw_data)
        return n_data
    
    def __getitem__(self, index : int):
        """Returns the entry at index from the dataset

        Args:
            index (int): the requested entry index of the dataset

        Returns:
            features (torch.Tensor): (N, d) Feature tensor of the selected slide with N patches and d feature dimensions
            label (int): Ground truth label {0, ..., n_classes}
            wsi_id (str): Name of the WSI as "DHMC_xxx" where xxx is a unique id of the slide (train == False only)
            coordinates (torch.Tensor): (N, 2) xy coordinates of the N patches of the selected slide (train == False only)
        """

        features = None
        label = None
        wsi_id = None
        coordinates = None
        
        # ------------------
        # Your code here ... 
        # ------------------
        features = torch.tensor(self.raw_data[index]["image_features"]).float()
        label = torch.tensor(self.raw_data[index]["label"])
        # coordinates = self.raw_data[index]["patch_coordinates"]
        # wsi_id = self.raw_data[index]["wsi_id"]
        # if self.train:
        return features, label
        # else:
        #     return features, label, wsi_id, coordinates

In [81]:
# Load all the img from output folder and make a dataset
coinDataset = CoinDataset('./output/', "train_data.json", train=True)
# trans = tranosforms.Compose([transforms.Resize(256),transforms.ToTensor()])
train_loader = DataLoader(coinDataset, batch_size=1, shuffle=True)
# dataset = datasets.ImaVgeFolder('./output-1/', transform=transforms.ToTensor())

In [91]:
from torchvision.models import resnet18, ResNet18_Weights

model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256, 16)
)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [92]:

def train(model, train_loader, optimizer, criterion, epochs=10):
    for epoch in range(epochs):
        for i, (data, target) in enumerate(train_loader):
            data = data.permute(0, 1, 2, 3)
            data, target = data.to(device), target.to(device)
            # data = data.view(-1, 256*256)
            optimizer.zero_grad()
            output = model(data)
            # output.softmax(0)
            # output = torch.round(output)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if i % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, i * len(data), len(train_loader.dataset),
                    100. * i / len(train_loader), loss.item()))
                

train(model, train_loader, torch.optim.Adam(model.parameters(), lr=1e-3), nn.CrossEntropyLoss(), epochs=10)

/var/folders/fr/v89fg7g56hb9fx7z3ymhhp4m0000gn/T/ipykernel_96376/1542035511.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(self.raw_data[index]["image_features"]).float()


Train Epoch: 0 [0/382 (0%)]	Loss: 2.387052
Train Epoch: 0 [10/382 (3%)]	Loss: 2.985427
Train Epoch: 0 [20/382 (5%)]	Loss: 2.560836
Train Epoch: 0 [30/382 (8%)]	Loss: 3.068016
Train Epoch: 0 [40/382 (10%)]	Loss: 2.210358
Train Epoch: 0 [50/382 (13%)]	Loss: 3.737090
Train Epoch: 0 [60/382 (16%)]	Loss: 2.537976
Train Epoch: 0 [70/382 (18%)]	Loss: 2.005456
Train Epoch: 0 [80/382 (21%)]	Loss: 1.725199
Train Epoch: 0 [90/382 (24%)]	Loss: 3.004172
Train Epoch: 0 [100/382 (26%)]	Loss: 2.685013
Train Epoch: 0 [110/382 (29%)]	Loss: 2.653685
Train Epoch: 0 [120/382 (31%)]	Loss: 2.124725
Train Epoch: 0 [130/382 (34%)]	Loss: 2.514086
Train Epoch: 0 [140/382 (37%)]	Loss: 2.653254
Train Epoch: 0 [150/382 (39%)]	Loss: 3.856172
Train Epoch: 0 [160/382 (42%)]	Loss: 2.245568
Train Epoch: 0 [170/382 (45%)]	Loss: 3.120234
Train Epoch: 0 [180/382 (47%)]	Loss: 2.832537
Train Epoch: 0 [190/382 (50%)]	Loss: 3.513852
Train Epoch: 0 [200/382 (52%)]	Loss: 3.723752
Train Epoch: 0 [210/382 (55%)]	Loss: 3.015148
Tra

KeyboardInterrupt: 